In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. In this notebook, we use ChatGPT, Gemini on GenerativeAI and Anthropic Claude.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message class allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool use

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

### ChatGPT

In [5]:
from langrila.openai import ToolConfig, ToolParameter, ToolProperty

chatgpt_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

ToolConfig object is internally converted to client tool configs

In [6]:
# You can see the formatted tool configs by running the following
chatgpt_tool_configs[1].format()

{'type': 'function',
 'function': {'name': 'start_music',
  'description': 'Play some music matching the specified parameters.',
  'parameters': {'type': 'object',
   'properties': {'energetic': {'type': 'boolean',
     'description': 'Whether the music is energetic or not.'},
    'loud': {'type': 'boolean',
     'description': 'Whether the music is loud or not.'},
    'bpm': {'type': 'number',
     'description': 'The beats per minute of the music.',
     'enum': [60, 180]}},
   'required': ['energetic', 'loud', 'bpm']}}}

In [7]:
chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
)

In [8]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party is on! 🌟 The disco ball is spinning, the music is pumping with an energetic beat at 180 BPM, and the lights are dimmed just right to set the mood. Let the fun begin! 🎉🕺💃')], name=None), usage=Usage(prompt_tokens=131, completion_tokens=51, total_tokens=182), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_gGsbkS9cuvTjFxKQOFmYfKOJ', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_usRCkN2ABRl67dMNCX0oFCse', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_pFrZZjM2J1sTXxX0Ccip6eLU', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_i

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party is on! 🌟 The disco ball is spinning, the music is pumping with an energetic beat at 180 BPM, and the lights are dimmed just right to set the mood. Let the fun begin! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 51},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_gGsbkS9cuvTjFxKQOFmYfKOJ',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_usRCkN2ABRl67dMNCX0oFCse',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_pFrZZjM2J1sTXxX0Ccip6eLU',
     'function': {'arguments': '{"brightness": 0.5}', 

Asynchronous generation

In [10]:
response = await chatgpt.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is officially on! 🎉✨ The disco ball is spinning, energetic music is pumping at 180 BPM, and the lights are dimmed for that perfect vibe. Let the fun begin! 🕺💃')], name=None), usage=Usage(prompt_tokens=131, completion_tokens=46, total_tokens=177), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_QQTaP6cLHhdTMjaX6OqJ9Jqv', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_MYSA4wsmbfmTne5Y4gU0YHsO', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_ked3DeWZxHmgmrDHaw45B05k', 'function': {'arguments': '{"brightness": 0.3}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': 'c

multiple generation

In [11]:
response = chatgpt.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party mood is on! \U0001faa9\n\n- The disco ball is spinning, casting dynamic lights all around!\n- Energetic music is blasting at 180 BPM, getting everyone pumped!\n- The lights are dimmed to a perfect vibe.\n\nGet ready to dance and have a great time! 🎉💃🕺'},
  {'text': 'The party mood is on! The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed perfectly for the vibe. Let’s get this celebration going! 🎉💃🕺'}],
 'name': None}

You can get the results from specific tool.

In [12]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "🎉 Let's crank up the tunes! The music is on with a high-energy vibe and a fast beat! \n\n🕺💃 Make sure to light up the space with some colorful lights and decorations. A few disco balls hanging from the ceiling would add to the party atmosphere!\n\n🍹 And don't forget the drinks—set up a fun drink station with some colorful cocktails or mocktails, and maybe some snacks that people can munch on while dancing.\n\n🎈 Get everyone involved with a dance-off or some fun games to keep the energy flowing! \n\nAre you ready to party? Let the good times roll! 🥳"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 60,
  'completion_tokens': 126},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_NSJ9TGB9YguyNefAGNu27O44',
     'fu

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chatgpt.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=163, completion_tokens=75, total_tokens=238), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_NgdhuB7ANy1ivu1leUpuZoH9', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_X5Llz6wDhIFhKRo7dBAiVj4A', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_XFxPI1tAODItzPb78ufHMmLi', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_NgdhuB7ANy1ivu1leUpuZoH9'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_X5Llz6w

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 163,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_NgdhuB7ANy1ivu1leUpuZoH9',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
     'call_id': 'call_X5Llz6wDhIFhKRo7dBAiVj4A',
     'args': '{"energetic": true, "loud": true, "bpm": 180}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_XFxPI1tAODItzPb78ufHMmLi',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_NgdhuB7ANy1ivu1leUpuZoH9'},
  

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_NgdhuB7ANy1ivu1leUpuZoH9', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_X5Llz6wDhIFhKRo7dBAiVj4A', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_XFxPI1tAODItzPb78ufHMmLi', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_NgdhuB7ANy1ivu1leUpuZoH9'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_X5Llz6wDhIFhKRo7dBAiVj4A'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_XFxPI1tAODItzPb78ufHMmLi')], name=None)

### Gemini on Google Generative AI

In [17]:
from langrila.gemini.genai import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is almost same as that of OpenAI
gemini_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "180", "240"],  # gemini does not support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [18]:
# You can see the formatted tool config as follows
gemini_tool_configs[1].format()

name: "start_music"
description: "Play some music matching the specified parameters."
parameters {
  type_: OBJECT
  properties {
    key: "loud"
    value {
      type_: BOOLEAN
      description: "Whether the music is loud or not."
    }
  }
  properties {
    key: "energetic"
    value {
      type_: BOOLEAN
      description: "Whether the music is energetic or not."
    }
  }
  properties {
    key: "bpm"
    value {
      type_: STRING
      description: "The beats per minute of the music."
      enum: "60"
      enum: "180"
      enum: "240"
    }
  }
  required: "energetic"
  required: "loud"
  required: "bpm"
}

In [19]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
)

In [20]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone! Let's get this party started! The disco ball is going, the music is pumping, and the lights are low. Time to let loose and have some fun! 🥳🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 143,
  'completion_tokens': 40},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "180"
  

Asynchronous generation

In [21]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "**BOOM!** Let's get this party started! The disco ball is spinning, the music is pumping, and the lights are low. Time to dance! 🎉 🎶 😄 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 143,
  'completion_tokens': 37},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "180"
          }
        }
     

Gemini allows you to call specific tool as well as ChatGPT.

In [22]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone, crank up the volume! Let's get this party started! 🎉🎶🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 66,
  'completion_tokens': 20},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "240"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! energetic=True loud=True, bpm=\'240\'"
          }
  

### Gemini on VertexAI

In [23]:
from langrila.gemini.vertexai import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is almost same as that of OpenAI
gemini_vertexai_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "180", "240"],  # gemini does not support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [24]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_vertexai_tool_configs,
)

In [25]:
response = gemini_vertexai.run("Turn this place into a party!")

In [26]:
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, the disco ball is spinning, the music is pumping, and the lights are dimmed! Let's get this party started! 🎉\n\nWhat kind of party are we throwing? What kind of music, decorations, and activities would make this place a blast?  Tell me more and I'll help you create the ultimate party atmosphere! 🎊 \n")], name=None), usage=Usage(prompt_tokens=71, completion_tokens=73, total_tokens=144), prompt=[role: "user"
parts {
  text: "Turn this place into a party!"
}
, role: "model"
parts {
  function_call {
    name: "power_disco_ball"
    args {
      fields {
        key: "power"
        value {
          bool_value: true
        }
      }
    }
  }
}
parts {
  function_call {
    name: "start_music"
    args {
      fields {
        key: "loud"
        value {
          bool_value: true
        }
      }
      fields {
        key: "energetic"
        value {
          bool_value: true
        }
      }
   

multiple generation

In [27]:
response = gemini_vertexai.run("Turn this place into a party!", n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "The disco ball is spinning, the music is pumping with an energetic beat, and the lights are dimmed to create the perfect party atmosphere. Let's get this party started! 🎉 \n"},
  {'text': "The disco ball is spinning, the music is pumping with an energetic beat, and the lights are dimmed to create the perfect party atmosphere. Let's get this party started! 🎉 \n"}],
 'name': None}

### Anthropic Claude

In [28]:
from langrila.claude import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is the same as that of OpenAI
claude_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [29]:
# You can see the formatted tool config as follows
claude_tool_configs[1].format()

{'name': 'start_music',
 'description': 'Play some music matching the specified parameters.',
 'input_schema': {'type': 'object',
  'properties': {'energetic': {'type': 'boolean',
    'description': 'Whether the music is energetic or not.'},
   'loud': {'type': 'boolean',
    'description': 'Whether the music is loud or not.'},
   'bpm': {'type': 'number',
    'description': 'The beats per minute of the music.',
    'enum': [60, 180]}},
  'required': ['energetic', 'loud', 'bpm']}}

In [30]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=claude_tool_configs,
)

In [31]:
prompt = "Turn this place into a party!"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up the perfect party atmosphere for you. Here's what I've done:\n\n1. Powered on the disco ball: The spinning disco ball will create a fun, festive ambiance with its reflective patterns around the room.\n\n2. Started the music: I've chosen energetic and loud music with a fast tempo of 180 beats per minute. This should get everyone in the mood to dance and have a great time.\n\n3. Adjusted the lighting: I've dimmed the lights to 60% brightness (0.6). This creates a nice balance between having enough light to see and move around safely, while still maintaining a party atmosphere that complements the disco ball.\n\nThese elements combined should create an exciting party environment. The spinning disco ball will scatter light around the room, the energetic music will encourage dancing, and the dimmed lights will set the right mood.\n\nIs there anything else you'd like me to adjust to make the party even better? I can

Asynchronous generation

In [32]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up the perfect party atmosphere for you. Here's what I've done:\n\n1. Powered on the disco ball: The spinning disco ball will create a fun, party-like atmosphere with its reflective patterns across the room.\n\n2. Started the music: I've chosen energetic and loud music with a fast tempo of 180 beats per minute. This should get everyone in the mood to dance and have a great time.\n\n3. Adjusted the lighting: I've dimmed the lights to 60% brightness (0.6). This creates a nice balance – dark enough for a party vibe, but still bright enough for people to see and move around safely.\n\nThese elements combined should create an exciting party atmosphere. The spinning disco ball, upbeat music, and dimmed lights will transform the space into a lively party setting.\n\nIs there anything else you'd like me to adjust to make the party even better? For example, we could change the music tempo or adjust the lighting further if

### Claude on Amazon Bedrock

In [33]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=claude_tool_configs,
)

In [34]:
prompt = "Turn this place into a party!"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The party atmosphere has been set up:\n\n1. The music has started playing. It's energetic and loud with a fast tempo of 180 beats per minute, perfect for dancing and creating a lively atmosphere.\n2. The lights have been dimmed to 30% brightness (0.3), creating a perfect party ambiance that's not too dark but definitely sets the mood.\n3. The disco ball is now powered on and spinning, adding that classic party vibe with its sparkling, moving lights.\n\nYour place is now ready for a party! The combination of upbeat music, dimmed lighting, and the spinning disco ball should create an exciting and fun atmosphere. Is there anything else you'd like to adjust or any other ways I can help enhance the party setup?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 975,
  'completion_tokens': 169},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a party!', 'typ

# Multi-turn conversation with tools

In [35]:
from langrila import InMemoryConversationMemory, JSONConversationMemory

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [36]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉 The lights are dimmed to a cool 50%, the disco ball is spinning, and the energetic music is pumping at 180 BPM. Let the festivities begin!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 41},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2NsCdgsvei7XZvWQzWKYCxKi',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_aTrnhgyLx5GiHjuEnSdtskY0',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_leetUNGPlESaYRdgwA86eTkJ',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_mus

Next turn

In [37]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The BPM has been cranked up to 240! The energy in here just shot through the roof. Time to dance! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 232,
  'completion_tokens': 30},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2NsCdgsvei7XZvWQzWKYCxKi',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_aTrnhgyLx5GiHjuEnSdtskY0',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_leetUNGPlESaYRdgwA86eTkJ',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_

## Multi-turn conversation using tools with multiple model

In [38]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
    conversation_memory=shared_memory,
)

In [39]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

In [40]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🌟 The disco ball is spinning, the energetic music is blasting at 180 bpm, and the lights are dimmed to create the perfect atmosphere. Let’s dance! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_YArNfwDAYbtqmktbUi9O1pT5',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_RtcTOwOq1iDYdP31dPSw5y3H',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_GymCCgfK8kKvhawKTrhzRobf',
     'function': {'arguments': '{"brightness": 0.5}', 'name':

Gemini can continue conversation without any additional process.

In [41]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Things just got even more intense! I've pushed the bpm up to a heart-pumping 240!  Get ready to move! ⚡🎶🔥 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 255,
  'completion_tokens': 35},
 'prompt': [parts {
    text: "Turn this place into a party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            number_value: 180
          }
        }
      }
    }
  }
  parts {
   

Gemini also works as well.

NOTE : Claude may not work in multi-turn conversation using tools. Bug fix is in progress.

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [42]:
from langrila import InMemoryConversationMemory

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [43]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="auto")

In [44]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [45]:
print(response.message.content[0].text)

The BPM has been increased to 240! Get ready for an even more energetic vibe! 🎶💥


In [46]:
response.usage

Usage(prompt_tokens=208, completion_tokens=22, total_tokens=230)

In [47]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_0EoJMVS9h3TxI9DmYZMLQvj5',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_6BiLsabDXcWhAj7iXGjr8K4w',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_ZNGnEEPKY62MzI9gtRNHinUF',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_0EoJMVS9h3TxI9DmYZMLQvj5',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_6BiLsabDXcWhAj7iXGjr8K4w',
  'name': 'start_music',
  'content': 'Starting music! energetic=True loud=True, bpm=180'},
 {'role': 'tool',
  'tool_call_id': 'call_ZNGnEEPKY62MzI9gtRNHinUF',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 'ass

In [48]:
# whole history
chatgpt.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_0EoJMVS9h3TxI9DmYZMLQvj5'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'call_id': 'call_6BiLsabDXcWhAj7iXGjr8K4w'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_ZNGnEEPKY62MzI9gtRNHinUF'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_0EoJMVS9h3TxI9DmYZMLQvj5',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
    'call_id': 'call_6BiLsabDXcWhAj7iXGjr8K4w',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'funcname': 'start_music'}],
  'name': None},
 {'role': 'function'